## Speaker Diatrization
Need to seperate audio by speaker. 
- https://www.youtube.com/watch?v=YRvf00NooN8

https://huggingface.co/pyannote/speaker-diarization-3.1

In [32]:
# Import necessary libraries
import os
from dotenv import load_dotenv
import torch
from pyannote.audio import Pipeline
import wave
import contextlib
import time

In [33]:
# Load environment variables
load_dotenv()

# Groq API setup
HF_API = os.getenv("HF_API")

In [34]:
# Define a simple timer function
def timer(label):
    class Timer:
        def __enter__(self):
            self.start = time.time()
            return self
        
        def __exit__(self, *args):
            end = time.time()
            print(f"{label}: {end - self.start:.2f} seconds")
    
    return Timer()

In [35]:
# Set up the pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token = HF_API)


In [36]:
# Send pipeline to GPU (when available)
if torch.cuda.is_available():
  pipeline.to(torch.device('cuda'))

In [37]:
# Define the audio file path
audio_file = "audio/elon_ted.wav"

In [38]:
# Get the duration of the audio file
with contextlib.closing(wave.open(audio_file,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)

In [39]:
# Run the diarization
with timer("Speaker Diarization"):
    diarization = pipeline(audio_file, num_speakers=None, min_speakers=1, max_speakers=5)

# Print the results
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

# Calculate and print speaker statistics
speaker_duration = {}
for segment, _, speaker in diarization.itertracks(yield_label=True):
    if speaker not in speaker_duration:
        speaker_duration[speaker] = segment.duration
    else:
        speaker_duration[speaker] += segment.duration

print("\nSpeaker Statistics:")
for speaker, duration in speaker_duration.items():
    percentage = (duration / duration) * 100
    print(f"{speaker}: {duration:.2f} seconds ({percentage:.2f}% of total)")

TypeError: 'NoneType' object is not callable

In [ ]:
# run the pipeline (with progress bar)
from pyannote.audio.pipelines.utils.hook import ProgressHook
with ProgressHook() as hook:
  diarization = pipeline(audio_file, hook=hook)

Output()

TypeError: 'NoneType' object is not callable